<H1> PROJET 3 : Réalisez une étude de santé publique </H1>



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<B> <span> 1ère question : CALCUL DE LA POPULATION MONDIALE  </span>  </B>

In [2]:
df_population_brut = pd.read_csv("FAOSTAT_data_POPULATION.csv")
df_population_brut.head()

Code Domaine              Domaine  Code Pays            Pays  Code Élément  \
0          FBS  Bilans Alimentaires          2     Afghanistan           511   
1          FBS  Bilans Alimentaires        202  Afrique du Sud           511   
2          FBS  Bilans Alimentaires          3         Albanie           511   
3          FBS  Bilans Alimentaires          4         Algérie           511   
4          FBS  Bilans Alimentaires         79       Allemagne           511   

             Élément  Code Produit     Produit  Code Année  Année  \
0  Population totale          2501  Population        2013   2013   
1  Population totale          2501  Population        2013   2013   
2  Population totale          2501  Population        2013   2013   
3  Population totale          2501  Population        2013   2013   
4  Population totale          2501  Population        2013   2013   

            Unité  Valeur Symbole Description du Symbole  
0  1000 personnes   30552     NaN      Donnée officielle  
1  1000 personnes   52776     NaN      Donnée officielle  
2  1000 personnes    3173     NaN      Donnée officielle  
3  1000 personnes   39208     NaN      Donnée officielle  
4  1000 personnes   82727     NaN      Donnée officielle

In [3]:
# Je ne garde que les colonnes qui m'intéressent
df_population = df_population_brut[['Pays','Unité','Valeur']]
df_population.columns = ['pays','unite','valeur']
# D'abord une 1ère analyse de mon DataFrame
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
pays      175 non-null object
unite     175 non-null object
valeur    175 non-null int64
dtypes: int64(1), object(2)
memory usage: 4.2+ KB


Il y a donc 175 lignes, je choisis comme clé primaire le Code pays, et je vais supprimer les doublons pour vérifier qu'il y a toujours le même nombre de ligne après

In [4]:
#je choisis comme clé primaires le pays et le vérifie en supprimant les doublons

df_population.drop_duplicates('pays').head()

pays           unite  valeur
0     Afghanistan  1000 personnes   30552
1  Afrique du Sud  1000 personnes   52776
2         Albanie  1000 personnes    3173
3         Algérie  1000 personnes   39208
4       Allemagne  1000 personnes   82727

In [5]:
#Je vérifie que le nombre de ligne est le même avant et après cette manipulation
df_population.size == df_population.drop_duplicates('pays').size

True

Il y a toujours 175 lignes donc je garde Code Pays comme clé primaire

In [6]:
# Je calcule la population mondiale en faisant la somme des différents pays
print(round(((df_population["valeur"].sum())/1000000),1), "milliard de personnes")
# je divise par 1 millions, car l'unité est de 1000 personnes, et je souhaite un résultats en milliards

8.4 milliard de personnes


En 2013, la population mondiale était d'à peu près 7 milliard, nous avons donc 1.4 milliard de trop


In [7]:
# Je recherche le pays qui présenterait une redondance

df_population.loc[df_population['pays'].str.contains('Chine')]

pays           unite   valeur
33    Chine - RAS de Hong-Kong  1000 personnes     7204
34        Chine - RAS de Macao  1000 personnes      566
35         Chine, continentale  1000 personnes  1385567
36   Chine, Taiwan Province de  1000 personnes    23330
174                      Chine  1000 personnes  1416667

In [8]:
# Je vérifie si "Chine"(index 174) est bien une agrégation des 4 régions
((df_population.iloc[[33,34,35,36], :])["valeur"].sum()) ==((df_population.iloc[[174], :])["valeur"].sum())

True

In [9]:
# Je décide donc de supprimer la ligne "Chine" pour garder les régions différenciées
df_population = df_population[df_population['pays'] != 'Chine']
# Je recalcule la population mondiale
print(round(((df_population["valeur"].sum())/1000000),1), "milliard de personnes")
# je divise par 1 millions, car l'unité et de 1000 personnes, et je souhaite un résultats en milliards

7.0 milliard de personnes


Nous sommes à 7 milliards d'humains, le chiffre est donc tout à fait probable.

<B> <span> 2ème question : IDENTIFIER LES REDONDANCES  </span>  </B>

disponibilité intérieure = production + importations - exportations + évolution des stocks

Mais je cherche aussi les emplois, je vais vérifier que l'on peut aussi dire : disponibilité intérieure = aliments pour animaux + semences + pertes + traitements + autres utilisations + nourriture

In [10]:
# J'importe deux nouveaux DataFrame végétaux et animaux en retirant l'agrégation "Chine"

df_bilan_vegetaux_brut = pd.read_csv("FAOSTAT_bilan_vegetaux.csv")
df_bilan_vegetaux_brut = df_bilan_vegetaux_brut[df_bilan_vegetaux_brut['Pays'] != 'Chine']

df_bilan_animaux_brut = pd.read_csv("FAOSTAT_bilan_animaux.csv")
df_bilan_animaux_brut = df_bilan_animaux_brut[df_bilan_animaux_brut['Pays'] != 'Chine']

# Je nettoie mes dataframe
df_bilan_vegetaux = df_bilan_vegetaux_brut[['Pays','Code Élément','Élément','Produit','Unité','Valeur']]
df_bilan_vegetaux.columns = ['pays','code_element','element','produit','unite','valeur']

df_bilan_animaux = df_bilan_animaux_brut[['Pays','Code Élément','Élément','Produit','Unité','Valeur']]
df_bilan_animaux.columns = ['pays','code_element','element','produit','unite','valeur']

# Avant de concaténer ces deux DataFrame, je leur assigne une nouvelle qualité : végétaux ou animaux
df_bilan_vegetaux=df_bilan_vegetaux.assign(groupe='vegetaux')
df_bilan_animaux=df_bilan_animaux.assign(groupe='animaux')

df_bilan_complet = pd.concat([df_bilan_vegetaux, df_bilan_animaux])
df_bilan_complet.head()

pays  code_element                   element produit  \
0  Afghanistan          5511                Production     Blé   
1  Afghanistan          5611   Importations - Quantité     Blé   
2  Afghanistan          5072        Variation de stock     Blé   
3  Afghanistan          5301  Disponibilité intérieure     Blé   
4  Afghanistan          5527                  Semences     Blé   

                unite  valeur    groupe  
0  Milliers de tonnes  5169.0  vegetaux  
1  Milliers de tonnes  1173.0  vegetaux  
2  Milliers de tonnes  -350.0  vegetaux  
3  Milliers de tonnes  5992.0  vegetaux  
4  Milliers de tonnes   322.0  vegetaux

In [11]:
# Je vérifie la formule "disponibilité intérieure = Production + importations - exportations + évolution des stocks"
# avec la France et le blé

df_bilan_complet.loc[(df_bilan_complet['pays'].str.contains('France')) & (df_bilan_complet['produit'].str.contains('Blé'))]

pays  code_element  \
36006  France          5511   
36007  France          5611   
36008  France          5072   
36009  France          5911   
36010  France          5301   
36011  France          5521   
36012  France          5527   
36013  France          5123   
36014  France          5131   
36015  France          5154   
36016  France          5142   
36017  France           645   
36018  France           664   
36019  France           674   
36020  France           684   

                                                 element produit  \
36006                                         Production     Blé   
36007                            Importations - Quantité     Blé   
36008                                 Variation de stock     Blé   
36009                            Exportations - Quantité     Blé   
36010                           Disponibilité intérieure     Blé   
36011                              Aliments pour animaux     Blé   
36012                                           Semences     Blé   
36013                                             Pertes     Blé   
36014                                         Traitement     Blé   
36015                                Autres Utilisations     Blé   
36016                                         Nourriture     Blé   
36017  Disponibilité alimentaire en quantité (kg/pers...     Blé   
36018     Disponibilité alimentaire (Kcal/personne/jour)     Blé   
36019  Disponibilité de protéines en quantité (g/pers...     Blé   
36020  Disponibilité de matière grasse en quantité (g...     Blé   

                    unite    valeur    groupe  
36006  Milliers de tonnes  38614.00  vegetaux  
36007  Milliers de tonnes   2055.00  vegetaux  
36008  Milliers de tonnes   1131.00  vegetaux  
36009  Milliers de tonnes  21502.00  vegetaux  
36010  Milliers de tonnes  20298.00  vegetaux  
36011  Milliers de tonnes   7822.00  vegetaux  
36012  Milliers de tonnes    748.00  vegetaux  
36013  Milliers de tonnes    358.00  vegetaux  
36014  Milliers de tonnes   1575.00  vegetaux  
36015  Milliers de tonnes   2824.00  vegetaux  
36016  Milliers de tonnes   6971.00  vegetaux  
36017                  kg    108.43  vegetaux  
36018  Kcal/personne/jour    811.00  vegetaux  
36019     g/personne/jour     25.99  vegetaux  
36020     g/personne/jour      4.20  vegetaux

In [12]:
#Disponibilité intérieure = Production + importations + évolution des stocks - exportations 

(df_bilan_complet.iloc[[36010], :]["valeur"].sum()) == ((df_bilan_complet.iloc[[36006,36007,36008], :])["valeur"].sum())-((df_bilan_complet.iloc[[36009], :])["valeur"])


36009    True
Name: valeur, dtype: bool

La formule est donc bien confirmée pour "disponibilité intérieure = Production + importations - exportations + évolution des stocks"

Maintenant essayons "disponibilité intérieure = aliments pour animaux + semences + pertes + traitements + autres utilisations + nourriture"

In [13]:
(df_bilan_complet.iloc[[36010], :]["valeur"].sum()) == ((df_bilan_complet.iloc[[36011,36012,36013,36014,36015,36016], :])["valeur"].sum())

True

<B> Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal </B>

In [14]:
# je veux créer un DataFrame ne contenant que la disponibilité alimentaire (Kcal/personne/jour) pour chaque produit

df_dispo_alim_kcal = df_bilan_complet[ df_bilan_complet['code_element'] == 664 ]
df_dispo_alim_kcal = df_dispo_alim_kcal[['pays','produit','valeur','groupe']]
df_dispo_alim_kcal.columns = ['pays','produit','kcal/pers/jour','groupe']
df_dispo_alim_kcal.head()

pays           produit  kcal/pers/jour    groupe
8   Afghanistan               Blé          1369.0  vegetaux
18  Afghanistan  Riz (Eq Blanchi)           141.0  vegetaux
30  Afghanistan              Orge            26.0  vegetaux
42  Afghanistan              Maïs            21.0  vegetaux
51  Afghanistan            Millet             3.0  vegetaux

In [15]:
# Et je crée un DataFrame df_dispo_alim_pop contenant la disponibilité alimentaire + la population pour chaque pays
df_dispo_alim_pop = pd.merge(df_dispo_alim_kcal, df_population)

df_dispo_alim_pop = df_dispo_alim_pop [['pays','produit', 'kcal/pers/jour', 'groupe','valeur']]
df_dispo_alim_pop.columns = ['pays','produit','kcal/pers/jour','groupe','milliers de pers']
df_dispo_alim_pop.head()

pays           produit  kcal/pers/jour    groupe  milliers de pers
0  Afghanistan               Blé          1369.0  vegetaux             30552
1  Afghanistan  Riz (Eq Blanchi)           141.0  vegetaux             30552
2  Afghanistan              Orge            26.0  vegetaux             30552
3  Afghanistan              Maïs            21.0  vegetaux             30552
4  Afghanistan            Millet             3.0  vegetaux             30552

In [16]:
# Je crée une nouvelle colonne nommée "Dispo Kcal/Jour" qui est la multiplication de la disponibilité alimentaire 
# et de la population

df_dispo_alim_pop['dispo kcal/Jour'] = (df_dispo_alim_pop['kcal/pers/jour'] * df_dispo_alim_pop['milliers de pers'] * 1000)

# Je supprime les ligne où le resultats est égal à 0
df_dispo_alim_pop = df_dispo_alim_pop[df_dispo_alim_pop['kcal/pers/jour'] != 0]

df_dispo_alim_pop.head()

pays           produit  kcal/pers/jour    groupe  milliers de pers  \
0  Afghanistan               Blé          1369.0  vegetaux             30552   
1  Afghanistan  Riz (Eq Blanchi)           141.0  vegetaux             30552   
2  Afghanistan              Orge            26.0  vegetaux             30552   
3  Afghanistan              Maïs            21.0  vegetaux             30552   
4  Afghanistan            Millet             3.0  vegetaux             30552   

   dispo kcal/Jour  
0     4.182569e+10  
1     4.307832e+09  
2     7.943520e+08  
3     6.415920e+08  
4     9.165600e+07

<B> Calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg </B>

In [17]:
# Il me faut donc le df_dispo_alim_kcal créé plus haut que je vais joindre à un DataFrame ne contenant que la 
# dispo alimentaire en quantité et créer une nouvelle colonne avec le ratio energie/poids

df_dispo_alim_kg = df_bilan_complet[ df_bilan_complet['code_element'] == 645 ]
df_dispo_alim_kg = df_dispo_alim_kg[['pays','produit','valeur','groupe']]
df_dispo_alim_kg.columns = ['pays','produit','dispo kg/pers/an','groupe']

df_dispo_alim_kg.head()

pays           produit  dispo kg/pers/an    groupe
7   Afghanistan               Blé            160.23  vegetaux
17  Afghanistan  Riz (Eq Blanchi)             13.82  vegetaux
29  Afghanistan              Orge              2.92  vegetaux
41  Afghanistan              Maïs              2.50  vegetaux
50  Afghanistan            Millet              0.40  vegetaux

In [18]:
df_ratio_energie_poids = pd.merge(df_dispo_alim_kcal, df_dispo_alim_kg)

# je retire les lignes où il y aurait un 0 avant de diviser
df_ratio_energie_poids = df_ratio_energie_poids[df_ratio_energie_poids['kcal/pers/jour'] != 0]
df_ratio_energie_poids = df_ratio_energie_poids[df_ratio_energie_poids['dispo kg/pers/an'] != 0]

df_ratio_energie_poids['ratio kcal/kg'] = (df_ratio_energie_poids['kcal/pers/jour']) / ((df_ratio_energie_poids['dispo kg/pers/an'])/365)

#Je nettoie le dataframe
df_ratio_energie_poids = df_ratio_energie_poids[['pays','produit','groupe','ratio kcal/kg']]
                                                 
# Je retire les lignes qui ont seulement la colonne ratio energie/poids = à NaN
df_ratio_energie_poids = df_ratio_energie_poids.dropna(subset = ['ratio kcal/kg'])

# et je retire les lignes où celui ci est égal à 0
df_ratio_energie_poids = df_ratio_energie_poids[df_ratio_energie_poids['ratio kcal/kg'] != 0]

df_ratio_energie_poids.sample(5)

pays              produit    groupe  ratio kcal/kg
5051              Jordanie               Pommes  vegetaux     467.093236
9881               Uruguay     Céréales, Autres  vegetaux    2147.058824
7908   République de Corée        Huile de Soja  vegetaux    8830.645161
11606              Géorgie  Viande de Volailles   animaux    1218.648208
6526               Myanmar               Manioc  vegetaux    1023.364486

<B>Donnez le ratio énergie/poids pour 5 produits de votre choix</B>

In [19]:
# Je calcule la moyenne par produit, donc pour 1Kg de produit nous avons le nombre de Kcal par Kg
df_ratio_energie_poids_total = df_ratio_energie_poids.groupby('produit')['ratio kcal/kg'].mean()
round(df_ratio_energie_poids_total.sample(5),1)

produit
Patates douces          967.5
Huile de Son de Riz    9206.1
Sucre, canne            298.9
Bananes                 617.0
Perciform               630.4
Name: ratio kcal/kg, dtype: float64

In [20]:
# Les 5 produits les plus caloriques
round(df_ratio_energie_poids_total.nlargest(),1)

produit
Huiles de Foie de Poisso    11809.0
Huile de Sésame             10146.1
Huile d'Olive                9292.7
Huile de Germe de Maïs       9257.1
Huile de Son de Riz          9206.1
Name: ratio kcal/kg, dtype: float64

In [21]:
# Les 5 produits les moins caloriques
round(df_ratio_energie_poids_total.nsmallest(),1)

produit
Citrons & Limes       194.8
Tomates               202.7
Mollusques, Autres    239.6
Pamplemousse          248.8
Agrumes, Autres       257.0
Name: ratio kcal/kg, dtype: float64

<B> Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal </B>

In [22]:
#Je reprend mon df_dispo_alim_pop, je ne garde que les végétaux et calcule la somme
df_dispo_vegetaux = df_dispo_alim_pop[df_dispo_alim_pop['groupe'] == 'vegetaux' ]

print(round(((df_dispo_vegetaux["dispo kcal/Jour"].sum()/1000000000000)),1), "billions de Kcal par jour")

16.6 billions de Kcal par jour


<B> Question : en termes de disponibilité intérieure de produits végétaux, combien d'humains pourraient être nourris si toute la production végétale était utilisée pour de la nourriture ? </B>

In [23]:
# Je crée un dataframe comprenant la production par pays et par produit (végétaux)
# à partir de mon dataframe df_bilan_vegetaux

df_prod_vegetaux = df_bilan_vegetaux[df_bilan_vegetaux['code_element'] == 5511]
df_prod_vegetaux = df_prod_vegetaux[df_prod_vegetaux['valeur'] != 0]
df_prod_vegetaux.sample(5)

pays  code_element     element  \
31395                                  Espagne          5511  Production   
95311                            Tchéquie (la)          5511  Production   
377                             Afrique du Sud          5511  Production   
101213  Venezuela (République bolivarienne du)          5511  Production   
3415                                    Angola          5511  Production   

        produit               unite  valeur    groupe  
31395      Maïs  Milliers de tonnes  4888.0  vegetaux  
95311    Raisin  Milliers de tonnes    75.0  vegetaux  
377         Blé  Milliers de tonnes  1870.0  vegetaux  
101213   Sésame  Milliers de tonnes    12.0  vegetaux  
3415    Bananes  Milliers de tonnes  3095.0  vegetaux

In [24]:
# Je fais une jointure avec mon dataframe contenant les ratio Kcal/Kg pour obtenir une colonne indiquant
# le nombre de calories disponible par pays et par produit :

df_prod_vegetaux_kcal = pd.merge(df_prod_vegetaux, df_ratio_energie_poids)
df_prod_vegetaux_kcal.head()

pays  code_element     element           produit  \
0  Afghanistan          5511  Production               Blé   
1  Afghanistan          5511  Production  Riz (Eq Blanchi)   
2  Afghanistan          5511  Production              Orge   
3  Afghanistan          5511  Production              Maïs   
4  Afghanistan          5511  Production            Millet   

                unite  valeur    groupe  ratio kcal/kg  
0  Milliers de tonnes  5169.0  vegetaux    3118.548337  
1  Milliers de tonnes   342.0  vegetaux    3723.950796  
2  Milliers de tonnes   514.0  vegetaux    3250.000000  
3  Milliers de tonnes   312.0  vegetaux    3066.000000  
4  Milliers de tonnes    13.0  vegetaux    2737.500000

In [25]:
df_prod_vegetaux_kcal['kcalories_produites/jour'] = (((df_prod_vegetaux_kcal['valeur']*1000000)/365) * df_prod_vegetaux_kcal['ratio kcal/kg'])

#Je nettoie le DataFrame
df_prod_vegetaux_kcal = df_prod_vegetaux_kcal[['pays','produit','groupe','kcalories_produites/jour']]
df_prod_vegetaux_kcal.sample(5)

pays                 produit    groupe  \
887   Chine, continentale          Épices, Autres  vegetaux   
2151             Jamaïque          Fruits, Autres  vegetaux   
2031                 Iraq                  Raisin  vegetaux   
1923                 Inde  Huile Graines de Coton  vegetaux   
3783         Sierra Leone       Bananes plantains  vegetaux   

      kcalories_produites/jour  
887               4.653846e+09  
2151              4.897959e+07  
2031              4.868239e+08  
1923              2.539759e+10  
3783              1.084058e+08

In [26]:
#KCalories disponibles dans le monde
print((df_prod_vegetaux_kcal['kcalories_produites/jour'].sum()/10**12), "billions de Kcal")

34.39555642480874 billions de Kcal


In [27]:
#Recommandation FAO de 2414 Kcal/jour (je divise le résultats pour avoir en milliards)

print(((df_prod_vegetaux_kcal['kcalories_produites/jour'].sum()/2414)/10**9),"milliards de personnes")

14.248366373160206 milliards de personnes


<B>Question : en termes de disponibilité intérieure de produits végétaux, combien d'humains pourraient être nourris si toute la production végétale( mis à part ses utilisations en semences, traitement, et autre utilisations) était utilisée pour de la nourriture ?<B/>

D'après la formule, "disponibilité intérieure = aliments pour animaux + semences + pertes + traitements + autres utilisations + nourriture", il faut donc que je soustraie les semences, le traitement et 
les autres utilisations

Je vais créer plusieurs dataframes pour additionner ensuite les aliments pour animaux + les pertes + la nourriture

In [28]:
# Les aliments pour animaux
df_alim_animaux_vegetaux = df_bilan_vegetaux[df_bilan_vegetaux['code_element'] == 5521]
df_alim_animaux_vegetaux = df_alim_animaux_vegetaux[['pays','produit','valeur','groupe']]
df_alim_animaux_vegetaux.columns = ['pays','produit','alim animaux en milliers de tonnes','groupe']
df_alim_animaux_vegetaux.head(1)

pays produit  alim animaux en milliers de tonnes    groupe
25  Afghanistan    Orge                               360.0  vegetaux

In [29]:
# Les pertes
df_pertes_vegetaux = df_bilan_vegetaux[df_bilan_vegetaux['code_element'] == 5123]
df_pertes_vegetaux = df_pertes_vegetaux[['pays','produit','valeur','groupe']]
df_pertes_vegetaux.columns = ['pays','produit','pertes en milliers de tonnes','groupe']
df_pertes_vegetaux.head(1)

pays produit  pertes en milliers de tonnes    groupe
5  Afghanistan     Blé                         775.0  vegetaux

In [30]:
# La nourriture
df_nourriture_vegetaux = df_bilan_vegetaux[df_bilan_vegetaux['code_element'] == 5142]
df_nourriture_vegetaux = df_nourriture_vegetaux[['pays','produit','valeur','groupe']]
df_nourriture_vegetaux.columns = ['pays','produit','nourriture en milliers de tonnes','groupe']
df_nourriture_vegetaux.head(1)

pays produit  nourriture en milliers de tonnes    groupe
6  Afghanistan     Blé                            4895.0  vegetaux

Je joins donc ces 3 dataframes en 1 et créé une nouvelle colonne qui les aditionne, puis j'ajoute les ratio kcal/kg pour avoir le nombre de kcal par pays et produits

In [31]:
DF1 = pd.merge(df_alim_animaux_vegetaux, df_pertes_vegetaux)
DF2 = pd.merge(DF1, df_nourriture_vegetaux)
df_prod_vegetaux_kcal_bis = pd.merge(DF2, df_ratio_energie_poids)

df_prod_vegetaux_kcal_bis.head()

pays              produit  alim animaux en milliers de tonnes  \
0     Afghanistan                 Orge                               360.0   
1     Afghanistan                 Maïs                               200.0   
2     Afghanistan  Légumineuses Autres                                 4.0   
3  Afrique du Sud                  Blé                                37.0   
4  Afrique du Sud                 Orge                                19.0   

     groupe  pertes en milliers de tonnes  nourriture en milliers de tonnes  \
0  vegetaux                          52.0                              89.0   
1  vegetaux                          31.0                              76.0   
2  vegetaux                           3.0                              74.0   
3  vegetaux                          74.0                            3174.0   
4  vegetaux                           7.0                               5.0   

   ratio kcal/kg  
0    3250.000000  
1    3066.000000  
2    3440.573770  
3    2986.529187  
4    4055.555556

In [32]:
df_prod_vegetaux_kcal_bis['alim_dispo/jour'] = (((df_prod_vegetaux_kcal_bis['alim animaux en milliers de tonnes']) + (df_prod_vegetaux_kcal_bis['pertes en milliers de tonnes']) + (df_prod_vegetaux_kcal_bis['nourriture en milliers de tonnes']))*1000000)/365
df_prod_vegetaux_kcal_bis = df_prod_vegetaux_kcal_bis[['pays','produit','groupe','ratio kcal/kg','alim_dispo/jour']]
df_prod_vegetaux_kcal_bis.head()

pays              produit    groupe  ratio kcal/kg  \
0     Afghanistan                 Orge  vegetaux    3250.000000   
1     Afghanistan                 Maïs  vegetaux    3066.000000   
2     Afghanistan  Légumineuses Autres  vegetaux    3440.573770   
3  Afrique du Sud                  Blé  vegetaux    2986.529187   
4  Afrique du Sud                 Orge  vegetaux    4055.555556   

   alim_dispo/jour  
0     1.372603e+06  
1     8.410959e+05  
2     2.219178e+05  
3     9.000000e+06  
4     8.493151e+04

In [33]:
df_prod_vegetaux_kcal_bis['kcalories_dispo/jour'] = ((df_prod_vegetaux_kcal_bis['alim_dispo/jour']) * df_prod_vegetaux_kcal_bis['ratio kcal/kg'])

#Je nettoie le DataFrame
df_prod_vegetaux_kcal_bis = df_prod_vegetaux_kcal_bis[['pays','produit','groupe','kcalories_dispo/jour']]
df_prod_vegetaux_kcal_bis.sample(5)

pays          produit    groupe  \
1065                                      Zambie           Millet  vegetaux   
141                           Bosnie-Herzégovine           Seigle  vegetaux   
62                                     Argentine  Légumes, Autres  vegetaux   
850   République populaire démocratique de Corée           Sorgho  vegetaux   
674                                      Myanmar           Sorgho  vegetaux   

      kcalories_dispo/jour  
1065          8.666667e+07  
141           9.512195e+07  
62            1.443396e+09  
850           2.932331e+08  
674           1.984932e+09

Je fais donc la somme de ces calories disponibles et je divise par le nombre de calories recommandées par la FAO

In [34]:
print(round(((df_prod_vegetaux_kcal_bis['kcalories_dispo/jour'].sum()/2414)/1000000000),1),"milliards de personnes")

6.9 milliards de personnes


<B>Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ? (Donnez ce résultat pour l'année 2013 ainsi que pour la dernière année disponible)<B/>

In [35]:
df_population_sousalim = pd.read_csv("FAOSTAT_pop_sousalim.csv")
df_population_sousalim = df_population_sousalim[['Zone','Année','Valeur']]
df_population_sousalim.columns = ['zone','annee','nbr pers sous alim en millions']
df_population_sousalim.head()

zone      annee  nbr pers sous alim en millions
0     Afghanistan  2012-2014                             7.2
1     Afghanistan  2015-2017                             NaN
2  Afrique du Sud  2012-2014                             2.0
3  Afrique du Sud  2015-2017                             NaN
4         Albanie  2012-2014                             0.2

In [36]:
# Je supprime la chine et les valeurs NaN
df_population_sousalim = df_population_sousalim[df_population_sousalim['zone'] != 'Chine']
df_population_sousalim = df_population_sousalim.dropna(subset = ['nbr pers sous alim en millions'])
df_population_sousalim.sample(5)

zone      annee  nbr pers sous alim en millions
94   Côte d'Ivoire  2012-2014                             3.3
268        Ouganda  2012-2014                            12.6
272       Pakistan  2012-2014                            37.8
32      Bangladesh  2012-2014                            25.8
350         Soudan  2012-2014                             9.9

In [37]:
# Je crée un dataframe pour 2013 et un autre pour 2016
df_population_sousalim_2013 = df_population_sousalim[df_population_sousalim['annee'] == '2012-2014']
df_population_sousalim_2016 = df_population_sousalim[df_population_sousalim['annee'] == '2015-2017']

In [38]:
# Je me rends compte qu'il n'y a pas de données pour 2016, je vérifie
df_population_sousalim.loc[df_population_sousalim['annee'].str.contains('2015-2017')]

Empty DataFrame
Columns: [zone, annee, nbr pers sous alim en millions]
Index: []

In [39]:
#Je teste la somme du dataframe complet et juste de celui de 2013
df_population_sousalim_2013['nbr pers sous alim en millions'].sum() == df_population_sousalim['nbr pers sous alim en millions'].sum()

True

Je ne garde donc que l'année 2013 et supprime la colonne indiquant l'année

In [40]:
df_population_sousalim_2013 = df_population_sousalim_2013[['zone','nbr pers sous alim en millions']]
df_population_sousalim_2013.sample()

zone  nbr pers sous alim en millions
224  Mali                             1.0

In [41]:
print(round(df_population_sousalim_2013['nbr pers sous alim en millions'].sum(),1), "millions de personnes sous-alimentés")

722.7 millions de personnes sous-alimentés


Donc la proportion de la population est :

In [42]:
print(round((df_population_sousalim_2013['nbr pers sous alim en millions'].sum()*100)/(df_population["valeur"].sum()/1000),1), "pourcent de personnes sous-alimentés")

10.3 pourcent de personnes sous-alimentés


In [43]:
# Les pays les plus touchés
df_population_sousalim_2013[df_population_sousalim_2013['nbr pers sous alim en millions'] > 20]

zone  nbr pers sous alim en millions
32            Bangladesh                            25.8
80   Chine, continentale                           142.4
122             Éthiopie                            27.8
168                 Inde                           195.1
272             Pakistan                            37.8

Etablissez la liste des produits (ainsi que leur code) considérés comme des céréales selon la FAO

Dans "Définitions et standards" du site de la FAO, les céréales à l'exception de ceux pour produire de la bière sont :

2516	Avoine ; 2511	Blé ; 2520	Céréales, Autres ; 2514	Maïs ; 2517	Millet ; 2513	Orge ; 2805	Riz (Eq Blanchi) ; 2515	Seigle ; 2518	Sorgho

In [44]:
# Les aliments pour animaux ont le code 5521, donc je crée un nouveau dataframe en sélectionnant les céréales
# qui sont produits pour les animaux

df_alim_cereales =df_bilan_vegetaux_brut[(df_bilan_vegetaux_brut['Code Produit'] == 2516) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2511) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2520) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2514) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2517) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2513) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2805) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2515) |\
                                         (df_bilan_vegetaux_brut['Code Produit'] == 2518)]

df_alim_animaux = df_alim_cereales[(df_alim_cereales['Code Élément'] == 5521)]

df_alim_animaux = df_alim_animaux[['Pays','Produit','Valeur']]
df_alim_animaux.columns = ['pays','produit','milliers de tonnes']
df_alim_animaux = df_alim_animaux[df_alim_animaux ['milliers de tonnes'] != 0]
df_alim_animaux.sample(5)

pays produit  milliers de tonnes
82102  République populaire démocratique de Corée  Seigle                21.0
69845                                     Norvège    Maïs               139.0
77431                                     Pologne    Maïs              2998.0
31400                                     Espagne    Maïs              9120.0
89039                                   Slovaquie  Avoine                12.0

A l'aide de la table donnant les ratios énergie/poids de chaque aliment (que vous avez établie précédemment), convertir ces données (filtrées) en kcal.

In [45]:
df_alim_animaux_ratio = pd.merge(df_alim_animaux, df_ratio_energie_poids)
df_alim_animaux_ratio['dispo Kcal'] = (df_alim_animaux_ratio['milliers de tonnes']*1000000)\
                                                    *(df_alim_animaux_ratio['ratio kcal/kg'])
df_alim_animaux_ratio.sample(5)

pays           produit  milliers de tonnes    groupe  \
442     Portugal            Seigle                 2.0  vegetaux   
405  Ouzbékistan               Blé              3274.0  vegetaux   
160     Équateur              Maïs               830.0  vegetaux   
231       Guyana  Riz (Eq Blanchi)                91.0  vegetaux   
263      Irlande              Orge              1380.0  vegetaux   

     ratio kcal/kg    dispo Kcal  
442    2588.019560  5.176039e+09  
405    2845.106132  9.314877e+12  
160    2902.409639  2.409000e+12  
231    3546.095586  3.226947e+11  
263    1766.129032  2.437258e+12

De la même manière que vous avez calculé la table des ratios énergie/poids , calculez les ratios poids de protéines/poids total, puis convertissez ces données (filtrées) en terme de protéines (unité : gramme).

In [46]:
# Je crée un dataframe avec la dispo alimentaire de protéines qui est le code 674
df_dispo_proteine = df_bilan_complet[(df_bilan_complet['code_element'] == 674)]

df_dispo_proteine = df_dispo_proteine[['pays','produit','valeur']]
df_dispo_proteine.columns = ['pays','produit','proteine g/pers/jour']

df_dispo_proteine = df_dispo_proteine[df_dispo_proteine ['proteine g/pers/jour'] != 0]

df_dispo_proteine.sample(5)

pays                  produit  proteine g/pers/jour
30758          Samoa  Poissons Marins, Autres                  1.41
17928          Japon             Cephalopodes                  1.54
33574  Tchéquie (la)            Viande, Autre                  1.95
6619       Australie                    Bière                  0.73
25836    Ouzbékistan       Poissons Eau Douce                  0.13

In [47]:
#Je reprends mon df de la dispo alimentaire en poids
df_dispo_alim_kg.head()

pays           produit  dispo kg/pers/an    groupe
7   Afghanistan               Blé            160.23  vegetaux
17  Afghanistan  Riz (Eq Blanchi)             13.82  vegetaux
29  Afghanistan              Orge              2.92  vegetaux
41  Afghanistan              Maïs              2.50  vegetaux
50  Afghanistan            Millet              0.40  vegetaux

In [48]:
#Je fais une jointure naturelle
df_ratio_proteine_poids = pd.merge(df_dispo_proteine,df_dispo_alim_kg)
df_ratio_proteine_poids.sample()

pays           produit  proteine g/pers/jour  dispo kg/pers/an  \
5867  Vanuatu  Céréales, Autres                  0.22              0.93   

        groupe  
5867  vegetaux

In [49]:
#je crée une nouvelle colonne pour créer ce ratio proteine/poids total et nettoie le dataframe
df_ratio_proteine_poids['ratio_proteine_par_g'] = (df_ratio_proteine_poids['proteine g/pers/jour'])/((df_ratio_proteine_poids['dispo kg/pers/an'])*1000/365)
df_ratio_proteine_poids.sample()

pays produit  proteine g/pers/jour  dispo kg/pers/an  \
4678  République de Moldova    Miel                  0.01               0.7   

        groupe  ratio_proteine_par_g  
4678  vegetaux              0.005214

In [50]:
df_ratio_proteine_poids = df_ratio_proteine_poids[['pays','produit','ratio_proteine_par_g','groupe']]
df_ratio_proteine_poids = df_ratio_proteine_poids[df_ratio_proteine_poids['ratio_proteine_par_g'] != 0]
df_ratio_proteine_poids.sample()

pays produit  ratio_proteine_par_g    groupe
795  Botswana     Blé              0.088957  vegetaux

In [51]:
# Je calcule la moyenne par produit, donc pour 1g de produit nous avons le nombre de protéine
df_ratio_proteine_poids_total = df_ratio_proteine_poids.groupby('produit')['ratio_proteine_par_g'].mean()
df_ratio_proteine_poids_total.sample()

produit
Crème    0.0303
Name: ratio_proteine_par_g, dtype: float64

In [52]:
# Les 5 produits les intéressants en terme de proteines
round(df_ratio_proteine_poids_total.nlargest(5),3)

produit
Graines Colza/Moutarde    0.365
Soja                      0.273
Arachides Decortiquees    0.251
Pois                      0.224
Légumineuses Autres       0.223
Name: ratio_proteine_par_g, dtype: float64

<B>Quelle proportion (en terme de poids) de la disponibilité mondiale de céréales est destinée à l'alimentation animale ? <B/>

In [53]:
#Il me faut un nouveau dataframe avec la dispo interieure des cereales
df_dispo_interieur_cereales = df_alim_cereales[(df_alim_cereales['Code Élément'] == 5301)]
df_dispo_interieur_cereales = df_dispo_interieur_cereales[['Pays','Produit','Valeur']]
df_dispo_interieur_cereales.columns = ['pays','produit','dispo_interieur_milliers_de_tonnes']
df_dispo_interieur_cereales = df_dispo_interieur_cereales[df_dispo_interieur_cereales['dispo_interieur_milliers_de_tonnes'] != 0]
df_dispo_interieur_cereales.sample()

pays           produit  dispo_interieur_milliers_de_tonnes
76790  Philippines  Céréales, Autres                                 8.0

In [54]:
print(round((df_alim_animaux['milliers de tonnes'].sum()*100)/\
            (df_dispo_interieur_cereales["dispo_interieur_milliers_de_tonnes"].sum()),1),\
      "pourcent de la disponibilité mondiale de céréales est destinée à l'alimentation animale")


36.3 pourcent de la disponibilité mondiale de céréales est destinée à l'alimentation animale


Avec l'aide des tables énergie/poids et poids de protéines/poids total, calculez la disponibilité mondiale totale de produit animaux, que vous exprimerez en kcal et en grammes de protéines.

In [55]:
# Je crée un dataframe de la dispo mondiale de produit animaux en kg en reprenant mon datataframe df_dispo_alim_kg
df_dispo_alim_kg.sample()

pays             produit  dispo kg/pers/an   groupe
35774  Vanuatu  Poissons Eau Douce              0.37  animaux

In [56]:
df_dispo_animaux = df_dispo_alim_kg[df_dispo_alim_kg['groupe'] == 'animaux' ]
df_dispo_animaux.sample()

# Je joins la table protéine/poids

pays              produit  dispo kg/pers/an   groupe
25455  Oman  Poissons Pelagiques              5.59  animaux

In [57]:
#Je joins la population pour calculer le poids total dispo

df_dispo_animaux = pd.merge(df_dispo_animaux,df_population)
df_dispo_animaux['dispo_totale_kg'] = (df_dispo_animaux['dispo kg/pers/an'])*((df_dispo_animaux['valeur'])*1000)
df_dispo_animaux.sample()

pays             produit  dispo kg/pers/an   groupe           unite  \
2268  Mexique  Mollusques, Autres              0.93  animaux  1000 personnes   

      valeur  dispo_totale_kg  
2268  122332      113768760.0

In [58]:
# je fais une jointure avec les df energie/poids et protéines/poids
df_dispo_animaux_kcal_proteines = pd.merge(df_dispo_animaux, df_ratio_energie_poids)
df_dispo_animaux_kcal_proteines = pd.merge(df_dispo_animaux_kcal_proteines, df_ratio_proteine_poids)
df_dispo_animaux_kcal_proteines.sample()

pays             produit  dispo kg/pers/an   groupe           unite  \
1375  Malawi  Lait - Excl Beurre              7.98  animaux  1000 personnes   

      valeur  dispo_totale_kg  ratio kcal/kg  ratio_proteine_par_g  
1375   16363      130576740.0     594.611529               0.03156

In [59]:
# Je créé les nouvelles colonnes disponibilités
df_dispo_animaux_kcal_proteines['dispo_totale_kcal'] = (df_dispo_animaux_kcal_proteines['dispo_totale_kg'])*(df_dispo_animaux_kcal_proteines['ratio kcal/kg'])
df_dispo_animaux_kcal_proteines['dispo_totale_proteine_g'] = ((df_dispo_animaux_kcal_proteines['dispo_totale_kg'])*1000)*(df_dispo_animaux_kcal_proteines['ratio_proteine_par_g'])
df_dispo_animaux_kcal_proteines.sample()

pays             produit  dispo kg/pers/an   groupe  \
404  Cabo Verde  Lait - Excl Beurre            115.64  animaux   

              unite  valeur  dispo_totale_kg  ratio kcal/kg  \
404  1000 personnes     499       57704360.0     631.269457   

     ratio_proteine_par_g  dispo_totale_kcal  dispo_totale_proteine_g  
404              0.033773       3.642700e+10             1.948845e+09

In [60]:
df_dispo_animaux_kcal_proteines = df_dispo_animaux_kcal_proteines[['pays','produit','dispo kg/pers/an','dispo_totale_kcal','dispo_totale_proteine_g','groupe']]
df_dispo_animaux_kcal_proteines = df_dispo_animaux_kcal_proteines[df_dispo_animaux_kcal_proteines['dispo_totale_kcal'] != 0]
df_dispo_animaux_kcal_proteines = df_dispo_animaux_kcal_proteines[df_dispo_animaux_kcal_proteines['dispo_totale_proteine_g'] != 0]
df_dispo_animaux_kcal_proteines.sample()

pays    produit  dispo kg/pers/an  dispo_totale_kcal  \
1334  Luxembourg  Perciform              8.46       2.514850e+09   

      dispo_totale_proteine_g   groupe  
1334              400441500.0  animaux

In [61]:
print(round((df_dispo_animaux_kcal_proteines['dispo_totale_kcal'].sum()/1000000000000),1),"billions de Kg disponibles mondialement")
print(round((df_dispo_animaux_kcal_proteines['dispo_totale_proteine_g'].sum()/1000000000000),1),"billions de grammes proteines disponibles mondialement")

1279.5 billions de Kg disponibles mondialement
81.9 billions de grammes proteines disponibles mondialement


Calculez les 3 valeurs du ratio suivant, lorsque les quantités sont exprimées en kg, puis en kcal, puis en grammes de protéines :
Disponibilité de céréales destinée à l'alimentation animale / Disponibilité animale destinée à l'alimentation humaine

In [62]:
#Je crée un nouveaux dataframe pour l'alimentation des animaux avec 
#une colonne dispo kg, dispo kcal et dispo protéine
df_dispo_alim_animale = pd.merge(df_alim_animaux,df_ratio_energie_poids )
df_dispo_alim_animale = pd.merge(df_dispo_alim_animale,df_ratio_proteine_poids)
df_dispo_alim_animale['dispo_en_kg'] = df_dispo_alim_animale['milliers de tonnes']*1000000
df_dispo_alim_animale['dispo_en_kcal'] = df_dispo_alim_animale['dispo_en_kg']*df_dispo_alim_animale['ratio kcal/kg']
df_dispo_alim_animale['dispo_en_g_proteine'] = (df_dispo_alim_animale['dispo_en_kg']*1000)*df_dispo_alim_animale['ratio_proteine_par_g']
df_dispo_alim_animale = df_dispo_alim_animale[['pays','produit','dispo_en_kg','dispo_en_kcal','dispo_en_g_proteine','groupe']]
df_dispo_alim_animale.head()

pays produit   dispo_en_kg  dispo_en_kcal  dispo_en_g_proteine  \
0     Afghanistan    Orge  3.600000e+08   1.170000e+12         3.555000e+10   
1     Afghanistan    Maïs  2.000000e+08   6.132000e+11         1.635200e+10   
2  Afrique du Sud     Blé  3.700000e+07   1.105016e+11         3.458789e+09   
3  Afrique du Sud    Orge  1.900000e+07   7.705556e+10         7.705556e+08   
4  Afrique du Sud    Maïs  4.715000e+09   1.475121e+13         3.766889e+11   

     groupe  
0  vegetaux  
1  vegetaux  
2  vegetaux  
3  vegetaux  
4  vegetaux

In [63]:
#Je crée un nouveaux dataframe pour la disponibilité animale pour alimentation humaine avec 
#une colonne dispo kg, dispo kcal et dispo protéine
df_dispo_alim_humaine = pd.merge(df_dispo_animaux,df_ratio_energie_poids )
df_dispo_alim_humaine = pd.merge(df_dispo_alim_humaine,df_ratio_proteine_poids)
df_dispo_alim_humaine['dispo_en_kg'] = df_dispo_alim_humaine['dispo_totale_kg']
df_dispo_alim_humaine['dispo_en_kcal'] = df_dispo_alim_humaine['dispo_en_kg']*df_dispo_alim_humaine['ratio kcal/kg']
df_dispo_alim_humaine['dispo_en_g_proteine'] = (df_dispo_alim_humaine['dispo_en_kg']*1000)*df_dispo_alim_humaine['ratio_proteine_par_g']
df_dispo_alim_humaine = df_dispo_alim_humaine[['pays','produit','dispo_en_kg','dispo_en_kcal','dispo_en_g_proteine','groupe']]
df_dispo_alim_humaine.head()

pays                 produit  dispo_en_kg  dispo_en_kcal  \
0  Afghanistan        Viande de Bovins  140233680.0   3.010900e+11   
1  Afghanistan  Viande d'Ovins/Caprins  150315840.0   3.456959e+11   
2  Afghanistan     Viande de Volailles   74852400.0   1.003633e+11   
3  Afghanistan           Viande, Autre   11609760.0   1.115148e+10   
4  Afghanistan        Abats Comestible   52549440.0   5.575740e+10   

   dispo_en_g_proteine   groupe  
0         2.107630e+10  animaux  
1         2.129933e+10  animaux  
2         9.701788e+09  animaux  
3         1.895752e+09  animaux  
4         8.586640e+09  animaux

In [64]:
print("pour 1 Kg de céréales destinée à l'alimentation animale, il y a",round(df_dispo_alim_humaine['dispo_en_kg'].sum()/df_dispo_alim_animale['dispo_en_kg'].sum(),1),"Kg de disponibilité animale destinée à l'alimentation humaine")

pour 1 Kg de céréales destinée à l'alimentation animale, il y a 1.4 Kg de disponibilité animale destinée à l'alimentation humaine


In [65]:
print("pour 1 Kcal de céréales destinée à l'alimentation animale, il y a",round(df_dispo_alim_humaine['dispo_en_kcal'].sum()/df_dispo_alim_animale['dispo_en_kcal'].sum(),1),"Kcal de disponibilité animale destinée à l'alimentation humaine")

pour 1 Kcal de céréales destinée à l'alimentation animale, il y a 0.5 Kcal de disponibilité animale destinée à l'alimentation humaine


In [66]:
print("pour 1 gramme de proteine de céréales destinée à l'alimentation animale, il y a",round(df_dispo_alim_humaine['dispo_en_g_proteine'].sum()/df_dispo_alim_animale['dispo_en_g_proteine'].sum(),1),"grammes de proteine animale destinée à l'alimentation humaine")

pour 1 gramme de proteine de céréales destinée à l'alimentation animale, il y a 1.5 grammes de proteine animale destinée à l'alimentation humaine


<B>Question : Combien de kg de céréales sont nécessaires pour produire 1kg de viande ?<B/>

In [67]:
#Je crée un data frame ne contenant que la viande
df_viande = (df_dispo_alim_humaine.loc[df_dispo_alim_humaine['produit'].str.contains('Viande')])

In [68]:
print (round((df_dispo_alim_animale['dispo_en_kg'].sum()/df_viande['dispo_en_kg'].sum()),1))

2.7


<B> Export des dataframe pour la base de donnée SQL <B/>

In [69]:
# Je récupère un nouveau fichier depuis le site de la FAO
population = pd.read_csv("FAOSTAT_population_SQL.csv")
population['population'] = population['Valeur']*1000
population = population[['Pays','Code Pays','Année','population']]
population.columns = ['pays','code_pays','année','population']
population.head()

#La clé primaire sera le code pays et l'année

pays  code_pays  année  population
0     Afghanistan          2   2010    28398000
1     Afghanistan          2   2011    29105000
2     Afghanistan          2   2012    29825000
3     Afghanistan          2   2013    30552000
4  Afrique du Sud        202   2010    51452000

In [70]:
# Je récupère deux nouveaux fichiers depuis le site de la FAO (2 pour pouvoir séparer végéteaux et animaux)

dispo_alim_vegetaux = pd.read_csv("FAOSTAT_dispo_alim_V_SQL.csv")
dispo_alim_animaux = pd.read_csv("FAOSTAT_dispo_alim_A_SQL.csv")

dispo_alim_vegetaux = dispo_alim_vegetaux.assign(groupe='vegetaux')
dispo_alim_animaux = dispo_alim_animaux.assign(groupe='animaux')
dispo_alim = pd.concat([dispo_alim_vegetaux, dispo_alim_animaux])

#Je créé des dataframe pour avoir les colonnes voulue
dispo_alim_t = dispo_alim[(dispo_alim['Code Élément'] == 5301)] # Disponibilité alimentaire
dispo_alim_t = dispo_alim_t[['Pays','Code Pays','Année','Produit','Code Produit','Valeur','groupe']]
dispo_alim_t['dispo_alim_tonnes'] = dispo_alim_t['Valeur']*1000
dispo_alim_t = dispo_alim_t[['Pays','Code Pays','Année','Code Produit','Produit','dispo_alim_tonnes','groupe']]
dispo_alim_t.columns = ['pays','code_pays','année','code_produit','produit','dispo_alim_tonnes','type_produit']

dispo_alim_kcal = dispo_alim[(dispo_alim['Code Élément'] == 664)] # Disponibilité alim en Kcal/personne/jour
dispo_alim_kcal = dispo_alim_kcal[['Pays','Code Pays','Année','Code Produit','Produit','Valeur','groupe']]
dispo_alim_kcal.columns = ['pays','code_pays','année','code_produit','produit','dispo_alim_kcal_p_j','type_produit']

dispo_alim_p = dispo_alim[(dispo_alim['Code Élément'] == 674)] # Disponibilité protéine en g/personne/jour
dispo_alim_p = dispo_alim_p[['Pays','Code Pays','Année','Code Produit','Produit','Valeur','groupe']]
dispo_alim_p.columns = ['pays','code_pays','année','code_produit','produit','dispo_prot','type_produit']

dispo_alim_g = dispo_alim[(dispo_alim['Code Élément'] == 684)] # Disponibilité matière grasse en g/personne/jour
dispo_alim_g = dispo_alim_g[['Pays','Code Pays','Année','Code Produit','Produit','Valeur','groupe']]
dispo_alim_g.columns = ['pays','code_pays','année','code_produit','produit','dispo_mat_gr','type_produit']


dispo_alim1 = pd.merge(dispo_alim_t, dispo_alim_kcal)
dispo_alim2 = pd.merge(dispo_alim1, dispo_alim_p)
dispo_alim = pd.merge(dispo_alim2, dispo_alim_g)
dispo_alim = dispo_alim[['pays', 'code_pays', 'année', 'produit', 'code_produit', 'dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr','type_produit']]
dispo_alim.sample(5)

#La clé primaire sera le code pays, l'année et le code_produit

pays  code_pays  année                 produit  code_produit  \
510     Algérie          4   2012     Oranges, Mandarines          2611   
14005  Lettonie        119   2011            Pamplemousse          2613   
5928   Colombie         44   2010                Haricots          2546   
40641     Yémen        249   2012  Viande d'Ovins/Caprins          2732   
37242  Portugal        174   2013               Crustacés          2765   

       dispo_alim_tonnes  dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  \
510            1054000.0                 22.0        0.30          0.19   
14005             3000.0                  1.0        0.01          0.00   
5928            158000.0                 29.0        1.92          0.14   
40641            99000.0                 24.0        1.67          1.84   
37242            20000.0                  4.0        0.89          0.04   

      type_produit  
510       vegetaux  
14005     vegetaux  
5928      vegetaux  
40641      animaux  
37242      animaux

In [71]:
#Nouveau fichier depuis le site de la FAO
equilibre_prod_brut = pd.read_csv("FAOSTAT_equilibre_SQL.csv")

dispo_int = equilibre_prod_brut[(equilibre_prod_brut['Code Élément'] == 5301)] # Disponibilité intérieure
dispo_int = dispo_int[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
dispo_int.columns = ['pays','code_pays','année','produit','code_produit','dispo_int']

alim_ani = equilibre_prod_brut[(equilibre_prod_brut['Code Élément'] == 5521)] # Aliments pour animaux
alim_ani = alim_ani[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
alim_ani.columns = ['pays','code_pays','année','produit','code_produit','alim_ani']

alim_semences = equilibre_prod_brut[(equilibre_prod_brut['Code Élément'] == 5527)] # Semences
alim_semences = alim_semences[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
alim_semences.columns = ['pays','code_pays','année','produit','code_produit','semences']

alim_pertes = equilibre_prod_brut[(equilibre_prod_brut['Code Élément'] == 5527)] # Pertes
alim_pertes = alim_pertes[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
alim_pertes.columns = ['pays','code_pays','année','produit','code_produit','pertes']

alim_transforme = equilibre_prod_brut[(equilibre_prod_brut['Code Élément'] == 5131)] # Transformés
alim_transforme = alim_transforme[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
alim_transforme.columns = ['pays','code_pays','année','produit','code_produit','transfo']

alim_nourriture = equilibre_prod_brut[(equilibre_prod_brut['Code Élément'] == 5142)] # nourriture
alim_nourriture = alim_nourriture[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
alim_nourriture.columns = ['pays','code_pays','année','produit','code_produit','nourriture']

alim_autre = equilibre_prod_brut[(equilibre_prod_brut['Code Élément'] == 5154)] # Autres utilisations
alim_autre  = alim_autre [['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
alim_autre.columns = ['pays','code_pays','année','produit','code_produit','autres_utilisations']

#Je fais une jointure gauche car je veux garder les produits "viandes" qui seraient éliminés avec les semences
# alim_ani, etc...

equilibre_prod1 = pd.merge(dispo_int,alim_ani, how = 'left')
equilibre_prod2 = pd.merge(equilibre_prod1,alim_semences, how='left')
equilibre_prod3 = pd.merge(equilibre_prod2,alim_pertes, how='left')
equilibre_prod4 = pd.merge(equilibre_prod3,alim_transforme, how='left')
equilibre_prod5 = pd.merge(equilibre_prod4,alim_nourriture, how='left')
equilibre_prod = pd.merge(equilibre_prod5,alim_autre, how='left')

# Je remplace les NaN par 0
equilibre_prod.fillna(0, inplace = True)

#Je vérifie
equilibre_prod.loc[equilibre_prod['produit'].str.contains('Viande')]

#La clé primaire sera le code_pays, l'année et le code_produit

pays  code_pays  année                 produit  code_produit  \
188       Afghanistan          2   2010        Viande de Bovins          2731   
189       Afghanistan          2   2011        Viande de Bovins          2731   
190       Afghanistan          2   2012        Viande de Bovins          2731   
191       Afghanistan          2   2013        Viande de Bovins          2731   
192       Afghanistan          2   2010  Viande d'Ovins/Caprins          2732   
193       Afghanistan          2   2011  Viande d'Ovins/Caprins          2732   
194       Afghanistan          2   2012  Viande d'Ovins/Caprins          2732   
195       Afghanistan          2   2013  Viande d'Ovins/Caprins          2732   
196       Afghanistan          2   2010     Viande de Volailles          2734   
197       Afghanistan          2   2011     Viande de Volailles          2734   
198       Afghanistan          2   2012     Viande de Volailles          2734   
199       Afghanistan          2   2013     Viande de Volailles          2734   
200       Afghanistan          2   2010           Viande, Autre          2735   
201       Afghanistan          2   2011           Viande, Autre          2735   
202       Afghanistan          2   2012           Viande, Autre          2735   
203       Afghanistan          2   2013           Viande, Autre          2735   
520    Afrique du Sud        202   2010        Viande de Bovins          2731   
521    Afrique du Sud        202   2011        Viande de Bovins          2731   
522    Afrique du Sud        202   2012        Viande de Bovins          2731   
523    Afrique du Sud        202   2013        Viande de Bovins          2731   
524    Afrique du Sud        202   2010  Viande d'Ovins/Caprins          2732   
525    Afrique du Sud        202   2011  Viande d'Ovins/Caprins          2732   
526    Afrique du Sud        202   2012  Viande d'Ovins/Caprins          2732   
527    Afrique du Sud        202   2013  Viande d'Ovins/Caprins          2732   
528    Afrique du Sud        202   2010        Viande de Suides          2733   
529    Afrique du Sud        202   2011        Viande de Suides          2733   
530    Afrique du Sud        202   2012        Viande de Suides          2733   
531    Afrique du Sud        202   2013        Viande de Suides          2733   
532    Afrique du Sud        202   2010     Viande de Volailles          2734   
533    Afrique du Sud        202   2011     Viande de Volailles          2734   
...               ...        ...    ...                     ...           ...   
61162          Zambie        251   2012        Viande de Suides          2733   
61163          Zambie        251   2013        Viande de Suides          2733   
61164          Zambie        251   2010     Viande de Volailles          2734   
61165          Zambie        251   2011     Viande de Volailles          2734   
61166          Zambie        251   2012     Viande de Volailles          2734   
61167          Zambie        251   2013     Viande de Volailles          2734   
61168          Zambie        251   2010           Viande, Autre          2735   
61169          Zambie        251   2011           Viande, Autre          2735   
61170          Zambie        251   2012           Viande, Autre          2735   
61171          Zambie        251   2013           Viande, Autre          2735   
61516        Zimbabwe        181   2010        Viande de Bovins          2731   
61517        Zimbabwe        181   2011        Viande de Bovins          2731   
61518        Zimbabwe        181   2012        Viande de Bovins          2731   
61519        Zimbabwe        181   2013        Viande de Bovins          2731   
61520        Zimbabwe        181   2010  Viande d'Ovins/Caprins          2732   
61521        Zimbabwe        181   2011  Viande d'Ovins/Caprins          2732   
61522        Zimbabwe        181   2012  Viande d'Ovins/Caprins          2732   
61523        Zimbabwe        181   2013  Viande 

In [72]:
# Je récupère un nouveau fichier depuis le site de la FAO
sous_nutrition = pd.read_csv("FAOSTAT_sous_alim_SQL.csv")
sous_nutrition = sous_nutrition[['Zone','Code zone','Année','Valeur']]
sous_nutrition.columns = ['pays','code_pays','année','nb_personnes']

#Je remplace la notation des années pour uniformiser tous les DataFrames
sous_nutrition.replace({'2009-2011': 2010, '2010-2012': 2011, '2011-2013': 2012, '2012-2014': 2013,\
                       '2013-2015': 2014, '2014-2016': 2015, '2015-2017': 2016}, inplace=True)

sous_nutrition.head()

#La clé primaire sera le code_pays et l'année

pays  code_pays  année  nb_personnes
0  Afghanistan          2   2010           6.2
1  Afghanistan          2   2011           6.4
2  Afghanistan          2   2012           6.8
3  Afghanistan          2   2013           7.2
4  Afghanistan          2   2014           7.5

In [73]:
population.to_csv("population.csv", index = False)
dispo_alim.to_csv("dispo_alim.csv", index = False)
equilibre_prod.to_csv("equilibre_prod.csv", index = False)
sous_nutrition.to_csv("sous_nutrition.csv", index = False)